# Knowledge Map Liquidity Sample

This requires:

- `camelot-py[base]`
- `openai`
- `pandas`

And also [pdftotext](https://en.wikipedia.org/wiki/Pdftotext)

## Part 1: "Framework"

In [1]:
# let's define a few helper utilities

from pathlib import Path
import hashlib
import subprocess
import json

text_cache = Path('cache')

def sha1(input_string):
    """Helper to hash input strings"""
    try:

        # Step 5: Create a new SHA-1 hash object
        hash_object = hashlib.sha1()

        # Step 6: Update the hash object with the bytes-like object
        hash_object.update(input_string.encode('utf-8'))

        # Step 7: Get the hexadecimal representation of the hash
        return hash_object.hexdigest()
    except Exception as e:
        raise ValueError(input_string) from e
        
def pdftotext(source: str, target: Path):
    """Extract text from PDF. Requires pdftotext binary in the path"""
    
    command = ["pdftotext", "-enc","UTF-8",source, str(target)]
    result = subprocess.run(command, capture_output=True, text=True)

    if result.returncode != 0:
        if Path(target).exists():
            Path(target).unlink()
        print(f"{source}: {result.stdout} {result.stderr}".strip())

def extract_text(source):
    """Get text from PDF. Cache results to avoid recomputation"""
    local = text_cache / sha1(source.name + " > text")

    if local.exists():
        return local
    print(f"extracting from {source}")
    pdftotext(source, local)
    return local

def get_page_text(file):
    """Split extracted text into multiple pages"""
    splits = extract_text(file).read_text().split("\f")

    pages = []
    for i, s in enumerate(splits):
        pages.append(((i + 1), s))
    return pages

In [2]:
from functools import wraps
import inspect
def stored(func):
    """
    implements nix-like durable memoisation of function results.

    Lazy way to avoid recomputing expensive calls. Expects results to be JSON-serializable
    """
    @wraps(func)
    def CACHE(*args, **kwargs):
        name = func.__name__
        meta = {}

        meta["name"] = name
        meta["func"] = inspect.getsource(func)
        meta["args"] = args
        meta["kwargs"] = kwargs

        js = json.dumps(meta)
        sha = hashlib.sha1(js.encode('utf-8'))

        digest = sha.hexdigest()

        path = text_cache / f"{digest}-{name}.json"

        if path.exists():
            with path.open('r') as r:
                cached = json.load(r)
            return cached["result"]
        result = func(*args, **kwargs)
        meta["result"] = result
        with path.open('w') as w:
            json.dump(meta, w)
        return result

    return CACHE

In [4]:

import camelot

@stored
def extract_csvs(pdf, page, method):
    """Extract tables from the specified PDF page as CSV"""
    try:
        sourced = camelot.read_pdf(str(pdf), flavor=method, pages=str(page))
        results = []
        for s in sourced:
            tbl = s.df.to_csv(index=False)
            results.append(tbl)
        return results
    except Exception as e:
        print(f'Error on {pdf}' + e)
        return []
    


In [6]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

@stored
def get_gpt(content, model="gpt-4-1106-preview", temperature=0, max_tokens=1000):
    """
    Cached call to GPT.
    """
    messages = [{"role": "user", "content": content}]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return completion.model_dump()


# Part 2: Extractors

We have two:

1. Simple extractor that determines company name for the annual report
2. More complex extractor that gets liquidity values from the tables in annual report

In [7]:
def get_company_name(txt):

    prompt = """
You are CFO-GPT. You read annual reports and identify the name of the company in that report. 

Respond only with the name of the company or business entity. Respond with an empty string, if none is found

```txt
$TXT
```""".strip() + "\n"
    
    filled = prompt.replace("$TXT", txt)
    result = get_gpt(filled)['choices'][0]['message']['content']
    return result

In [8]:
# Hey, GPTExtractMasterBot. I want to extract from tables in annual reports information about the liquidity. 

# What are the names of the tables that will contain this value? Focus only on the most likely and common table names. Answer with a python function that checks if string contains a table that is likely to contain liquidity information


liquidity_tables = [
    'balance sheet',
    'cash flow statement',
    'cash flows'
]

keywords = [
    'cash and cash equivalents',
     'cash flow from operating activities',
    'free cash flow',
    'liquid assets'
]

def contains_liquidity_data(txt):
    """
    Check if the provided string contains a table name that is likely to contain liquidity information.

    Args:
    table_name (str): The name of the table to check.

    Returns:
    bool: True if the table_name is likely to contain liquidity information, False otherwise.
    """
    # Lowercase the table_name to make the search case-insensitive
    txt = txt.lower()
    
    # Define a list of common liquidity-related table names

    
    # Check if the table_name contains any of the liquidity-related table names
    table_found = any(s in txt for s in liquidity_tables)
    cell_found = any(s in txt for s in keywords)

    return table_found and cell_found


In [9]:
def fail_table(df) -> str:
    """Quick heuristic to check if extracted data frame looks valid"""
    txt = df.to_csv()


    if "...." in txt:
        return "looks like index"

    shape = df.shape

    surface = shape[0] * shape[1]
    if surface < 7:
        return f"Surface is only {surface}"

    ratio = len(txt) / surface

    if ratio > 40:
        return f"Text to cell ratio is {ratio}"

    return ""

In [10]:
def extract_liquidity_per_year(tables):

    prompt = """
    You are CFO-GPT. You read annual reports, looking for the amount of avalable liquidity (only cash and similar equivalents) in the annual reports.
    
    Pay attention to the units of measure of the tables. Return values as decimal number with a currency at the end.
    
    Answer with a json dictionary: `"year": "Value UnitOfMeasurement Currency"`. If value for the year is missing, then skip it.
    
    Do not write any code! Start your answer with "```json".
    
    # Source data to extract from
    
    Important! Pay attention to the units of measurement in each text chunk (usually at the beginning or the end of the chunk). 
    
    $CSV
    
    """.strip() + "\n"
    
    
    csv_joined = "\n".join(tables)
    
    
    filled = prompt.replace("$CSV", csv_joined)
    
    result = get_gpt(filled)['choices'][0]['message']['content']

    try:
    
        data = json.loads(result.replace("```json", "").replace("```", ""))
    except:
        raise ValueError(result)
    return data


In [19]:
import pandas as pd
from io import StringIO

sources = Path(r"C:\Users\MGroup\components_agent_sales\notebooks\famaga\tiny-data").rglob("*.pdf")

# print(list(sources))

knowledge_map = []

for pdf in sources:
    print(f"Reading {pdf}")
    tables = []

    raw_pages = get_page_text(pdf)

    company_name = get_company_name(raw_pages[0][1] + "\n" + raw_pages[1][1])
    
    print(f"  Company name: {company_name}")

    print(f"  Extracting liquidity")    
    pages = []    
    for num, txt in raw_pages:
        if contains_liquidity_data(txt):
            pages.append((num, txt))

    print(f"    interesting pages: {', '.join([str(t[0]) for t in pages])}")
    
            
    
    final = []
    for page, txt in pages:
        for method in ['stream', 'lattice']:
            found = False

            csvs = extract_csvs(str(pdf), page, method)

            for csv in csvs:
                df = pd.read_csv(StringIO(csv))
                fail_reason = fail_table(df)
                if fail_reason:
                    print(fail_reason)
                    continue
                final.append(f"\n```csv\n{csv}\n```\n\n")
    print(f"    found {len(final)} tables")
          

    if not final:
        print(f"Nothing found for {pdf}")
        continue
    liquidity = extract_liquidity_per_year(final)

    

    record = {"company": company_name, "liquidity": liquidity}
    print("Company map: " + json.dumps(record, indent=2, ensure_ascii=False))

    knowledge_map.append(record)



Reading C:\Users\MGroup\components_agent_sales\notebooks\famaga\tiny-data\Bellevue Group AG - Annual_Report_2022.pdf


AttributeError: 'str' object has no attribute 'name'

In [16]:
# take a look at the final knowledge map
knowledge_map

[{'company': 'Bellevue Group',
  'liquidity': {'2022': '64681 CHF', '2021': '84363 CHF'}},
 {'company': 'UNIQA Insurance Group AG',
  'liquidity': {'2022': '667.675 Thousand €', '2021': '592.583 Thousand €'}},
 {'company': 'Christian Dior',
  'liquidity': {'2022': '7.588 Billion EUR',
   '2021': '8.122 Billion EUR',
   '2020': '20.358 Billion EUR'}}]

In [17]:
prompt = """
You are CFO-GPT. Quickly answer, which of the companies has more liquidity, and how much? Don't make up information, if you are not certain.

```json
$MAP
```
"""

filled = prompt.replace("$MAP", json.dumps(knowledge_map))
result = get_gpt(filled)['choices'][0]['message']['content']
print(result)

To determine which company has more liquidity, we need to compare the liquidity figures for the most recent year provided, which is 2022. However, we need to ensure that we are comparing the values in the same currency. The liquidity for Bellevue Group is given in CHF (Swiss Francs), for UNIQA Insurance Group AG in thousands of EUR (Euros), and for Christian Dior in billions of EUR.

First, we need to convert the liquidity of Bellevue Group and UNIQA Insurance Group AG to the same scale as Christian Dior, which is in billions of EUR. However, without the current exchange rates, we cannot make an accurate conversion. Since I cannot make up information, I will compare the numbers as they are, but please note that this comparison will not reflect the true liquidity due to the difference in currency values and scales.

- Bellevue Group: 64,681 CHF
- UNIQA Insurance Group AG: 667.675 thousand EUR (which is 0.667675 billion EUR)
- Christian Dior: 7.588 billion EUR

Based on the numbers provi